In [ ]:
import numpy as np
import pandas as pd
import re
import os
import cv2
import random
import matplotlib.pyplot as plt
import gc
!pip install sklearn

TODO: 
- More features
- Resize images locally, then update the databases
- If all else fails, remove all but one video

In [ ]:
# Heatmap ... Frame#/Video#

heatMaps = {}
# 
heatmap_folder = os.listdir('../input/heatmapsv20/heatMapCSVs')
for video_heatmap_folder in heatmap_folder:
    videoId = re.search('\d+$', video_heatmap_folder)
    videoId = videoId.group(0)
    heatmap_folder_path = os.path.join('../input/heatmapsv20/heatMapCSVs', video_heatmap_folder)
    if videoId == '24':
        for heatmap in os.listdir(heatmap_folder_path):
            frameId = heatmap[:-4] #Remove .csv extension
            heatmap_path = os.path.join(heatmap_folder_path, heatmap)
            heatmapDataframe = pd.read_csv(heatmap_path)
            heatmapDataframe = heatmapDataframe.drop(heatmapDataframe.columns[0], axis=1)
            heatmapArray = heatmapDataframe.to_numpy()
            #heatmapTuple = (heatMapArray, frameId, videoId)
            key = videoId + '/' + frameId
            key.replace(" ", "")
            heatMaps[key] = (heatmapArray, videoId, frameId)

frame_paths = []

frame_folder = os.listdir('../input/sourceframesv20/SourceFrames')
for video_frame_folder in frame_folder:
    videoId = re.search('\d+$', video_frame_folder) 
    videoId = videoId.group(0)
    frame_folder_path = os.path.join('../input/sourceframesv20/SourceFrames', video_frame_folder)
    if videoId == '24':
        for frame in os.listdir(frame_folder_path):
            frameId = frame[5:-4] #Remove .jpg extension
            heatmap_path = os.path.join(frame_folder_path, frame)
            #frameTuple = (frame, frameId, videoId)
            frame_paths.append((heatmap_path, videoId, frameId))



In [ ]:
random.shuffle(frame_paths)

In [ ]:
nrows = 500
ncolumns = 250
channels = 10

def process_frame_and_heatmap(list_of_frames):
    X = [] #Images
    y = [] #Heatmaps
    
    #DEBUG ONLY
    n = len(list_of_frames)
    progress = 0
    count = 0
    
    for frameTuple in list_of_frames:
        frame, videoId, frameId = frameTuple
        img = cv2.cvtColor(cv2.imread(frame), cv2.COLOR_BGR2RGB)
        #Get features
        cannyed = cv2.Canny(img, 200, 225)
        count += 1
        
        im1 = np.array(cannyed)
        im2 = np.array(img)        
        im3 = np.zeros_like(im2)
        im4 = np.zeros_like(im2)
        gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        corners = cv2.goodFeaturesToTrack(gray, 25, 0.1, 10)
        corners = np.int0(corners)
        for i in corners:
            xx,yy = i.ravel()
            cv2.circle(im3, (xx, yy), 15, (255, 255, 255), -1, 8, shift=0)
        orb = cv2.ORB_create(200, 2.0)
        kp = orb.detect(gray, None)
        kp, des = orb.compute(gray, kp)
        im4 = cv2.drawKeypoints(gray, kp, im4, color=(255,0,0), flags=0)
        
        #Combine features
        combined = np.dstack((im1, im2, im3, im4)) 
        #print(combined) 
            
            
        #Get frameId and videoId
        key = videoId + '/' + frameId
        key.replace(" ", "")
        if not key in heatMaps:
            progress += (100 / n)
            continue
        X.append(combined)
        heatMap = heatMaps[key]
        heatMap = heatMap[0]
        
        y.append(heatMap)
        
        progress += (100 / n)
        print("Progress " + str(int(progress)) + "%", end='\r', flush=True)
    print()
    return X, np.array(X), np.array(y)

In [ ]:
test_frame_path = random.choice(frame_paths)
test_frame, test_videoId, test_frameId = test_frame_path

test = [test_frame_path]
a, test_processed, _ = process_frame_and_heatmap(test)

test_key = test_videoId + '/' + test_frameId
test_key.replace(" ", "")
test_heatmap_array, _, _ = heatMaps[key]

In [ ]:
vX, X, y = process_frame_and_heatmap(frame_paths)


plt.figure(figsize=(20,10))
columns = 5
for i in range(columns):
    plt.subplot(5 / columns + 1, columns, i + 1)
    plt.imshow(vX[-i][:, :, 1])
    print(y[-i])

In [ ]:
print(X.shape)

In [ ]:
#Lets split the data into train and test set
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20)

In [ ]:
del X
del y
gc.collect()

Model (Starting with previous semester's model)

In [ ]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential # to create a cnn model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Reshape
from keras.optimizers import RMSprop,Adam
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

model = Sequential()

model.add(Conv2D(filters = 48, kernel_size = (10,10), padding = 'Same', activation = 'relu', input_shape = (nrows, ncolumns, channels)))
# model.add(Conv2D(filters = 32, kernel_size = (6,6), padding = 'Same', activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 128, kernel_size = (5,5), padding = 'Same', activation = 'relu'))
# model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
# model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2), strides=(2,2)))
# model.add(Dropout(0.25))

# fully connected
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(5 * 10)) #Size of Heatmap
#model.add(BatchNormalization())
model.add(Reshape((5,10)))

#model.add(Dense(clusters * 2))

In [ ]:
model.summary()

In [ ]:
from keras.losses import MeanSquaredError

model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.001))
model.fit(X_train,y_train,epochs=8, validation_data=(X_val, y_val))

In [ ]:
model.evaluate(x=X_val, y=y_val)

In [ ]:
import seaborn as sb
from PIL import Image

predict = model.predict(test_processed)

val1 = np.array(predict[0])
plt.figure(figsize=(8,4),dpi=100)
ax = sb.heatmap(val1, vmin=0, cbar=False)
ax.axis('off')



In [ ]:
val1 = test_heatmap_array
plt.figure(figsize=(8,4),dpi=100)
ax = sb.heatmap(val1, vmin=0, cbar=False)
ax.axis('off')

In [ ]:
print(test_heatmap_array)
print(np.rint(np.array(predict[0])))

In [ ]:
from IPython.display import Image
Image(filename=test_frame) 

In [ ]:
X_val[0]